In [27]:
#@title Select your network details - Best nodes

import numpy as np
import pandas as pd
import json
import textwrap

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output
from urllib.parse import urlencode
from urllib.parse import quote

import plotly.graph_objects as go

def dropdown_menu_widget_healthy_aging():
    
    output = widgets.Output()

    dropdown_group = widgets.Dropdown(options = ['Bibliographic Coupling', 'Citation', 'Co-authorship', 'Co-citation'], value=None, description='Network Group:')
    dropdown_sub_category = widgets.Dropdown(options = ['Authors','Countries', 'Documents', 'Sources'], value='Authors', description='Sub_category:')
    dropdown_year = widgets.Dropdown(options = ['Until_2019', 2020, 2021, 2022], 
                                               value=2020, description='Year:')
    
    global url

    url = "https://raw.githubusercontent.com/SisonkeBiotik-Africa/AfriBioML/main/Network/{group}%20-%20{val_sub_category}/{val_year}.json"

    def output_by_state(val_group, val_sub_category, val_year):
        group = quote(val_group)
        url1 = url.format(group=group, val_sub_category=val_sub_category, val_year=val_year)
        #f"https://raw.githubusercontent.com/SisonkeBiotik-Africa/AfriBioML/main/Network/{group}%20-%20{val_sub_category}/{str(val_year)}.json"
        # make the analysis
        def make_analysis (url_par):
            data = pd.read_json(url_par, encoding='utf-8-sig', orient='records')
            full_data = pd.DataFrame.from_dict(data.iloc[0]['network'])
            final_df = pd.concat([full_data, pd.json_normalize(full_data['weights'])], axis=1)
            if 'scores' in final_df:
                final_df = final_df.drop(['weights', 'scores'], axis=1)
            else:
                final_df = final_df.drop('weights', axis=1)
            return final_df
        
        
        try:
            df1 = make_analysis(url1)
            if 'Total link strength' in df1.columns:
                display(df1.sort_values('Total link strength', ascending=False)   .head(15))  
            else:
                display(df1.head(6))

           
        except Exception as err:
            print("Location does not Exist")

            raise err

        # other 2 
        # print(url.format(group=group, val_sub_category=val_sub_category, val_year=2021))
        # df2 = make_analysis(url.format(group=group, val_sub_category=val_sub_category, val_year=2021))
        # if 'Total link strength' in df2.columns:
        #         display(df2.sort_values('Total link strength', ascending=False)   .head(6))  
        # else:
        #     display(df2  .head(6))
        # print(df2.head())
        # df3 = make_analysis(url.format(group=group, val_sub_category=val_sub_category, val_year=2022))
        # df3.head()


    def dropdown_group_eventhandler(change):
        """
        Eventhandler for the state dropdown widget
        """
        display(input_widgets)
        state_choice = change.new
        output_by_state(state_choice, dropdown_sub_category.value, dropdown_year.value)
        IPython.display.clear_output(wait=True)            

    def dropdown_sub_category_eventhandler(change):
        """
        Eventhandler for the question dropdown widget
        """
        display(input_widgets)
        question_choice = change.new
        output_by_state(dropdown_group.value, question_choice, dropdown_year.value)
        IPython.display.clear_output(wait=True)

    def dropdown_year_eventhandler(change):     
        """
        Event handler for the year dropdown widget
        """
        display(input_widgets)
        stratification_choice = change.new
        output_by_state(dropdown_group.value, dropdown_sub_category.value, stratification_choice)
        IPython.display.clear_output(wait=True)
            
    dropdown_group.observe(dropdown_group_eventhandler, names='value')
    dropdown_sub_category.observe(dropdown_sub_category_eventhandler, names='value')
    dropdown_year.observe(dropdown_year_eventhandler, names='value')

    
    input_widgets = widgets.HBox([dropdown_group, dropdown_sub_category, dropdown_year])
    
    display(input_widgets)
    IPython.display.clear_output(wait=True)  

dropdown_menu_widget_healthy_aging()

,id,label,x,y,cluster,Links,Total link strength,Documents,Citations,Norm. citations
821,3249,stein d.j.,0.6169,-0.3697,3,318.0,349.0,5.0,208.0,14.7810
670,2768,piras f.,0.4428,-0.3372,3,233.0,241.0,2.0,62.0,4.4059
61,450,banaj n.,0.4875,-0.3404,3,233.0,241.0,2.0,62.0,4.4059
921,3527,wang z.,0.4663,-0.3211,3,233.0,241.0,4.0,102.0,7.2484
974,3650,yun j.-y.,0.5176,-0.3266,3,233.0,241.0,2.0,62.0,4.4059
382,1737,james a.,0.4689,-0.2922,3,233.0,241.0,2.0,62.0,4.4059
54,378,assogna f.,0.4989,-0.3026,3,233.0,241.0,2.0,62.0,4.4059
814,3235,spalletta g.,0.4149,-0.3184,3,233.0,241.0,2.0,62.0,4.4059
459,1995,kwon j.s.,0.4372,-0.2981,3,233.0,241.0,2.0,62.0,4.4059
918,3520,wang l.,-0.1126,0.3794,1,229.0,229.0,2.0,29.0,2.0608


In [26]:
#@title Select your network details - Best edges

import numpy as np
import pandas as pd
import json
import textwrap

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output
from urllib.parse import urlencode
from urllib.parse import quote

import plotly.graph_objects as go

def dropdown_menu_widget_healthy_aging():
    
    output = widgets.Output()

    dropdown_group = widgets.Dropdown(options = ['Bibliographic Coupling', 'Citation', 'Co-authorship', 'Co-citation'], value=None, description='Network Group:')
    dropdown_sub_category = widgets.Dropdown(options = ['Authors','Countries', 'Documents', 'Sources'], value='Authors', description='Sub_category:')
    dropdown_year = widgets.Dropdown(options = ['Until_2019', 2020, 2021, 2022], 
                                               value=2020, description='Year:')
    
    global url

    url = "https://raw.githubusercontent.com/SisonkeBiotik-Africa/AfriBioML/main/Network/{group}%20-%20{val_sub_category}/{val_year}.json"

    def output_by_state(val_group, val_sub_category, val_year):
        group = quote(val_group)
        url1 = url.format(group=group, val_sub_category=val_sub_category, val_year=val_year)
        #f"https://raw.githubusercontent.com/SisonkeBiotik-Africa/AfriBioML/main/Network/{group}%20-%20{val_sub_category}/{str(val_year)}.json"
        # make the analysis
        def make_analysis (url_par):
            data = pd.read_json(url_par, encoding='utf-8-sig', orient='records')
            full_data = pd.DataFrame.from_dict(data.iloc[1]['network'])
            full_data_1 = pd.DataFrame.from_dict(data.iloc[0]['network'])
            dict01 = full_data_1.set_index('id').to_dict()["label"]  
            full_data["source_name"] = full_data["source_id"].apply(lambda x: dict01[x])
            full_data["target_name"] = full_data["target_id"].apply(lambda x: dict01[x])
            return full_data
        
        
        try:
            df1 = make_analysis(url1)
            if 'strength' in df1.columns:
                display(df1.sort_values('strength', ascending=False)   .head(15))  
            else:
                display(df1  .head(6)) 

           
        except Exception as err:
            print("Location does not Exist")

            raise err

        # other 2 
        # print(url.format(group=group, val_sub_category=val_sub_category, val_year=2021))
        # df2 = make_analysis(url.format(group=group, val_sub_category=val_sub_category, val_year=2021))
        # if 'Total link strength' in df2.columns:
        #         display(df2.sort_values('Total link strength', ascending=False)   .head(6))  
        # else:
        #     display(df2  .head(6))
        # print(df2.head())
        # df3 = make_analysis(url.format(group=group, val_sub_category=val_sub_category, val_year=2022))
        # df3.head()


    def dropdown_group_eventhandler(change):
        """
        Eventhandler for the state dropdown widget
        """
        display(input_widgets)
        state_choice = change.new
        output_by_state(state_choice, dropdown_sub_category.value, dropdown_year.value)
        IPython.display.clear_output(wait=True)            

    def dropdown_sub_category_eventhandler(change):
        """
        Eventhandler for the question dropdown widget
        """
        display(input_widgets)
        question_choice = change.new
        output_by_state(dropdown_group.value, question_choice, dropdown_year.value)
        IPython.display.clear_output(wait=True)

    def dropdown_year_eventhandler(change):     
        """
        Event handler for the year dropdown widget
        """
        display(input_widgets)
        stratification_choice = change.new
        output_by_state(dropdown_group.value, dropdown_sub_category.value, stratification_choice)
        IPython.display.clear_output(wait=True)
            
    dropdown_group.observe(dropdown_group_eventhandler, names='value')
    dropdown_sub_category.observe(dropdown_sub_category_eventhandler, names='value')
    dropdown_year.observe(dropdown_year_eventhandler, names='value')

    
    input_widgets = widgets.HBox([dropdown_group, dropdown_sub_category, dropdown_year])
    
    display(input_widgets)
    IPython.display.clear_output(wait=True)  

dropdown_menu_widget_healthy_aging()

,source_id,target_id,strength,source_name,target_name
175918,42375,47555,3575.0,"saba, l.","suri, j.s."
82388,18834,47555,1637.0,"gupta, d.","suri, j.s."
118847,25135,47555,1493.0,"khanna, n.n.","suri, j.s."
82336,18834,42375,1450.0,"gupta, d.","saba, l."
118796,25135,42375,1325.0,"khanna, n.n.","saba, l."
7751,2716,47555,1247.0,"araki, t.","suri, j.s."
188224,47553,47555,1188.0,"suri, h.s.","suri, j.s."
108576,22703,47555,1144.0,"jamthikar, a.d.","suri, j.s."
7721,2716,42375,1100.0,"araki, t.","saba, l."
175916,42375,47553,1076.0,"saba, l.","suri, h.s."
